# Store_avg_demand_query_in_pd

In [3]:
%load_ext sql
%load_ext lab_black

In [4]:
import sqlite3
import pandas as pd

## Connect to Database

In [5]:
from src.features import paths

path_to_data_database = paths.get_path_to_data("data.db")
path_to_data_database

'C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\processed\\data.db'

In [6]:
%sql sqlite:///$path_to_data_database

'Connected: @C:\\Users\\RowanM\\Documents\\dublin_energy_masterplan\\Residential\\data\\processed\\data.db'

In [7]:
conn = sqlite3.connect(path_to_data_database)

conn

## Find Residential IDs using Allocation file

In [11]:
path_to_allocations = paths.get_path_to_data("SME and Residential allocations.xlsx")
allocations = pd.read_excel(path_to_allocations, usecols=[0, 1], index_col=[0])

allocations.head()

,Code
ID,
1000,3
1001,3
1002,1
1003,1
1004,1


In [12]:
mask = allocations["Code"] == 1
residential_IDs = allocations[mask]

residential_IDs.head()

,Code
ID,
1002,1
1003,1
1004,1
1005,1
1006,1


In [32]:
select_IDs = tuple(residential_IDs.index)
select_IDs

(1002,
 1003,
 1004,
 1005,
 1006,
 1008,
 1009,
 1013,
 1014,
 1015,
 1016,
 1018,
 1020,
 1022,
 1024,
 1025,
 1027,
 1030,
 1033,
 1034,
 1035,
 1036,
 1037,
 1039,
 1041,
 1042,
 1044,
 1045,
 1047,
 1052,
 1053,
 1054,
 1055,
 1057,
 1058,
 1059,
 1060,
 1061,
 1062,
 1064,
 1065,
 1067,
 1068,
 1069,
 1071,
 1073,
 1074,
 1075,
 1076,
 1077,
 1078,
 1079,
 1080,
 1081,
 1082,
 1083,
 1084,
 1085,
 1086,
 1088,
 1091,
 1093,
 1094,
 1095,
 1096,
 1099,
 1102,
 1105,
 1107,
 1108,
 1109,
 1110,
 1112,
 1115,
 1119,
 1120,
 1121,
 1122,
 1123,
 1124,
 1126,
 1130,
 1132,
 1133,
 1134,
 1136,
 1138,
 1139,
 1140,
 1141,
 1143,
 1144,
 1147,
 1148,
 1152,
 1153,
 1155,
 1156,
 1157,
 1159,
 1160,
 1161,
 1164,
 1165,
 1166,
 1168,
 1169,
 1170,
 1171,
 1174,
 1176,
 1178,
 1179,
 1180,
 1182,
 1183,
 1184,
 1185,
 1191,
 1196,
 1197,
 1198,
 1199,
 1200,
 1201,
 1202,
 1203,
 1207,
 1210,
 1212,
 1214,
 1217,
 1219,
 1220,
 1222,
 1224,
 1225,
 1227,
 1229,
 1230,
 1231,
 1232,
 1233,

## Query Database

In [29]:
%%time

## Pull out all Residential IDs and find the average consumption

query = f"""
        SELECT Date, Time, AVG(Consumption) FROM electricity_data_ordered
        WHERE ID in {select_IDs}
        GROUP BY Date, Time;
        """

df = pd.read_sql_query(query,conn)

df.head()

Wall time: 9min 1s


,Date,Time,AVG(Consumption)
0,2009-07-15,00:00:00,0.450944
1,2009-07-15,00:30:00,0.357523
2,2009-07-15,01:00:00,0.294295
3,2009-07-15,01:30:00,0.251444
4,2009-07-15,02:00:00,0.225337


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25726 entries, 0 to 25725
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              25726 non-null  object 
 1   Time              25726 non-null  object 
 2   AVG(Consumption)  25726 non-null  float64
dtypes: float64(1), object(2)
memory usage: 603.1+ KB


In [31]:
%store df

Stored 'df' (DataFrame)
